In [1]:
pip install dash

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install dash-leaflet

Note: you may need to restart the kernel to use updated packages.


In [3]:
from dash import Dash, dcc, html
from dash.dependencies import Input, Output, State
import plotly.express as px
import dash_leaflet as dl


In [4]:
pip install --upgrade dash jupyter-dash


Note: you may need to restart the kernel to use updated packages.


Function definition (to be removed, just 1 in the code)

In [5]:
import requests
import pandas as pd
import geopandas as gpd
from shapely import wkt
import ipywidgets as widgets
import matplotlib.pyplot as plt
import folium
from shapely.geometry import mapping
#import leafmap as leafmap
from shapely.geometry import shape

def read_response(t):
    try:
        data = t.json() #This will convert the response to a json object
        return data
    except requests.exceptions.JSONDecodeError:
        print("Risposta non valida JSON!")
        print("Contenuto della risposta:", t.text)
        data = None 

def get_measurement_unit(pollutant):
    t=requests.post(url="http://127.0.0.1:5000/api/units", json={"var_pollutant": pollutant}) #json= data will convert the dictionary to a json object and send it to the server
    data = read_response(t) 
    return data[0]
    




# Dashboard

Populate the dropdown

In [6]:
# Function for both DV 10 and DV 11
def pollutant_dropdown():
    t=requests.get(url="http://127.0.0.1:5000/api/pollutant")
    list_pollutant = read_response(t)
    list = [{'label': pollutant, 'value': pollutant} for pollutant in list_pollutant]
    return list



This first part is just for the layout of the elements

In [7]:
# Get pollutants
#t = requests.get(url="http://127.0.0.1:5000/api/pollutant")
#list_pollutant = read_response(t)

# Get data for initial pollutant (DV 10)
def get_data_10(pollutant):
    data = {"var_pollutant": pollutant}
    t = requests.post(url="http://127.0.0.1:5000/api/DV_10", json=data)
    data = read_response(t)
    # to visualize the response, we can conver the data to a pandas geodataframe
    gdf = gpd.GeoDataFrame(data)
    gdf['geometry'] = gdf['geometry'].apply(wkt.loads)
    gdf.set_geometry('geometry', inplace=True)

    gdf = gdf.dropna(subset=['quota']) # Drop rows where 'quota' is NaN

    gdf['month'] = pd.to_datetime(gdf['month'], errors='coerce')
    gdf['month'] = gdf['month'].dt.strftime('%b %Y')
    return gdf

# Get data for initial pollutant (DV 11)
def get_data_11(pollutant):
    data = {"var_pollutant": pollutant}
    t = requests.post(url="http://127.0.0.1:5000/api/DV_11", json=data)
    data = read_response(t)
    df = pd.DataFrame(data)
    df['month'] = pd.to_datetime(df['month'], errors='coerce')
    df['month'] = df['month'].dt.strftime('%b %Y')
    return df


In [8]:
# contour lines file (DV 10)
import json
with open("../DATA/contour_lines.geojson", "r") as f:
    geojson_data = json.load(f)

#ADD colour to geojson based on elevation

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

def get_color(elev, min_elev=500, max_elev=3000):
    # Define a color map transitioning smoothly across elevations
    cmap = mcolors.LinearSegmentedColormap.from_list("terrain", [
        (0.0, "lawngreen"),   # Low elevation (green)
        (0.3, "yellow"),  # Mid elevation (yellow)
        (0.6, "orange"),  # High elevation (orange)
        (1.0, "brown")      # Highest elevation (red)
    ])

    # Normalize elevation to [0, 1] range
    ratio = (elev - min_elev) / (max_elev - min_elev)
    ratio = min(1, max(0, ratio))  # Clamp between 0 and 1

    # Convert colormap color to HEX format
    rgb = cmap(ratio)[:3]  # Get RGB values
    return f"#{int(rgb[0] * 255):02x}{int(rgb[1] * 255):02x}{int(rgb[2] * 255):02x}"

# Supponiamo tu abbia geojson_data come dizionario o FeatureCollection
for feature in geojson_data["features"]:
    elev = feature["properties"].get("ELEV", 1000)
    color = get_color(elev)
    feature["properties"]["color"] = color

print(geojson_data["features"][0])  # check the features structure

{'type': 'Feature', 'properties': {'fid': 1, 'ELEV': 2200.0, 'color': '#ec8b08'}, 'geometry': {'type': 'LineString', 'coordinates': [[10.237773054012042, 46.63439685859683], [10.237771002440088, 46.63430688921537], [10.237555801129059, 46.63427234207511], [10.237425411671927, 46.63428339632108]]}}


In [11]:

import dash
from dash import dcc, html, Output, Input, State
import pandas as pd
import geopandas as gpd
import requests

import plotly.express as px

import branca.colormap as cm
import dash_leaflet as dl

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from dash_extensions.javascript import assign
from dash.exceptions import PreventUpdate


# --- Dash App ---
app = dash.Dash(__name__)

#DV 10 functions --------------------------------------------------
# Style function for contour lines 

style = assign("""
function(feature) {
    return {
        color: feature.properties.color || "blue",
        weight: 1,
        opacity: 0.8
    };
}
""")

# DV 11 functions --------------------------------------------------
def get_years(df):
    return sorted(df["month"].str[-4:].unique())

def get_months(df, year):
    months = sorted(df[df["month"].str.contains(str(year[0]))]["month"].str[:3].unique(),
                    # key to get the months in order
                    key=lambda m: ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"].index(m))
    return ["All"] + months

# LAYOUT 
app.layout = html.Div([
    # DV 10
    html.Div([
        html.H3("DV 10 - Correlation map between monthly average of pollutant and station height"),
        html.Label("Pollutant:"),
        dcc.Dropdown(
            id='pollutant-dropdown_dv_10_11',
            options=pollutant_dropdown(),
            style={'margin-top': 20, 'margin-bottom': '20'},
        ),
        html.Div([
            html.Button('Visualize/update elevation map', id='el_map_dv_10', n_clicks=0, style={'margin-top': 20, 'margin-bottom': 20, 'padding': '6px 12px', 'fontSize': '15px'})
        ], style={'textAlign': 'center'}),

        # Loading wrapper for the map
        dcc.Loading(
            type="dot",  
            children=dl.Map(
                id="map",
                center=[45.64, 9.60],
                zoom=7,
                children=[dl.TileLayer(), dl.LayersControl(id="layers-control_dv_10")],
                style={"opacity": 0.7, "height": "50vh"},
            )
        )
    ]),

    html.Hr(style={"border": "1px solid lightgreen", "opacity": 0.7, "margin": "30px 0"}),  # Divider

    # DV 11
    html.Div([
        html.H3("DV 11 - Correlation between monthly average of pollutant and station height"),
        html.Label("Select between available years of the pollutant chosen in DV 10:"),
        dcc.Slider(id='year-slider', step=1.0),
        dcc.Dropdown(id='month-dropdown'),
        html.Div([
            html.Button('Visualize/update plot', id='el_plot_dv_11', n_clicks=0, style={'margin-top': 20, 'margin-bottom': 20, 'padding': '6px 12px', 'fontSize': '15px'})
        ], style={'textAlign': 'center'}),

        # Loading wrapper for the scatter plot
        dcc.Loading(
            type="dot",
            children=dcc.Graph(id='scatter-plot', style={'margin-top': '20px'})
        )
    ]),
], style={'padding': 20, 'margin': 2, 'border': '2px solid lightgreen', 'border-radius': '10px'})

# DV 10 callbacks ---------------------------------------------------------------------------
@app.callback(
    Output("layers-control_dv_10", "children"),
    Input("el_map_dv_10", "n_clicks"),
    State("pollutant-dropdown_dv_10_11", "value")    
)
def update_layers_dv_10(n_clicks, selected_pollutant):
    if n_clicks == 0:
      return []
    
    if n_clicks > 0 and not selected_pollutant:
        return [
                dl.TileLayer(),
                dl.Popup(
                    position=[45.64, 9.60],  # Central position
                    children=html.Div(
                        "Select a pollutant to visualize the map",
                        style={"backgroundColor": "red", "color": "white", "padding": "10px", "borderRadius": "5px"}
                    ))
            ]
    
    gdf = get_data_10(selected_pollutant)
    unit = get_measurement_unit(selected_pollutant)
    colormap_c = cm.LinearColormap(["blue", "lightblue", "yellow", "orange", "red"], vmin=gdf['monthly_average'].min(), vmax=gdf['monthly_average'].max())
    gdf["color"] = gdf["monthly_average"].apply(lambda x: colormap_c(x)[:7])

    return [
        dl.Overlay(
                dl.GeoJSON(
                    data=geojson_data,  # Path to the GeoJSON file
                    zoomToBounds=True,
                    options={"style":style}
                 ),
                name="Contour lines",
                checked=True
            ),

            # Overlay dinamici per ogni anno
            *[
                dl.Overlay(
                    dl.LayerGroup([
                        dl.CircleMarker(
                            center=[row.geometry.y, row.geometry.x],
                            radius=5,
                            color=row["color"],
                            fill=False,
                            fillOpacity=0.8,
                            children=[dl.Tooltip(f"{row['month']}: {row['monthly_average']} {unit}")]
                        )
                        for _, row in gdf[gdf["month"].str.contains(str(year))].iterrows()
                    ]),
                    name=f"{year}",
                    checked=(year == get_years(gdf)[-2])  # default most recent year (-1) checked
                )
                for year in get_years(gdf)
            ]
    ]



# DV 11 calbacks --------------------------------------------------------------------------
@app.callback(
    [Output('year-slider', 'min'),
     Output('year-slider', 'max'),
     Output('year-slider', 'value'),
     Output('year-slider', 'marks'),
     Output('month-dropdown', 'options'),
     Output('month-dropdown', 'value')],
    Input('pollutant-dropdown_dv_10_11', 'value')
)
def update_years_months(pollutant):
    if not pollutant:
        return (0, 0, 0, {}, [], 'not set')
    df = get_data_11(pollutant)
    years = get_years(df)
    months = get_months(df, years[0])

    return (float(years[0]), float(years[-1]), float(years[0]), {y: y for y in years},
            [{'label': m, 'value': m} for m in months], months[0]) #Default month set to ALL

@app.callback(
    Output('scatter-plot', 'figure'),
    Input("el_plot_dv_11", "n_clicks"),
    State('pollutant-dropdown_dv_10_11', 'value'), 
    State('year-slider', 'value'),
    State('month-dropdown', 'value')
)
def update_plot(n_clicks, pollutant, year, month):
    if n_clicks == 0:
      return px.scatter ()  # Return an empty figure if no button has been clicked
        
    if n_clicks > 0 and (not pollutant or not year or month is None):
        return px.scatter().update_layout(
            annotations=[dict(
                    text="Please select a pollutant, year and month to visualize the plot",
                    xref="paper", yref="paper",
                    showarrow=False,
                    font=dict(size=14, color="white"),
                    bgcolor="red",
                    borderpad=5)])
    df = get_data_11(pollutant)
    unit = get_measurement_unit(pollutant)
    df_filtered = df[df["month"].str.contains(str(year))]

    if month != "All":
        df_selected = df_filtered[df_filtered["month"].str.startswith(month)]
        df_other = df_filtered[~df_filtered["month"].str.startswith(month)]
    else:
        df_selected = df_filtered
        df_other = pd.DataFrame()
    fig = px.scatter(
        df_selected,
        x='monthly_average',
        y='quota',
        color='monthly_average',
        color_continuous_scale='RdYlBu_r',
        labels={
            'monthly_average': f"Monthly Average per pollutant {unit}",
            'quota': "Height [m]"
        },
        title=f"Correlation between monthly average of {pollutant} and station height"
    ).update_traces(marker=dict(size=8))

    fig.update_coloraxes(showscale=False)  # Remove color bar
    if not df_other.empty:
        fig.add_scatter(
            x=df_other['monthly_average'],
            y=df_other['quota'],
            mode='markers',
            marker=dict(color='grey', opacity=0.2),
            name='Other months'
        )
    fig.update_layout(showlegend=False)
    return fig



# In Jupyter, use run_server not run()
print("Starting Dashboard...")
app.run(port=8089)

Starting Dashboard...
